In [24]:
import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

     # Load embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Load FAISS indices - try both with and without _medibot suffix
symptom_index_path = "../indices/faiss_symptom_index"
if os.path.exists("../indices/faiss_symptom_index_medibot"):
    symptom_index_path = "../indices/faiss_symptom_index_medibot"

severity_index_path = "../indices/faiss_severity_index"
if os.path.exists("../indices/faiss_severity_index_medibot"):
    severity_index_path = "../indices/faiss_severity_index_medibot"

faiss_symptom = FAISS.load_local(
    symptom_index_path, embeddings,
    allow_dangerous_deserialization=True
)
faiss_severity = FAISS.load_local(
    severity_index_path, embeddings,
    allow_dangerous_deserialization=True
)


In [26]:
def test_symptom_combinations(faiss_symptom, faiss_severity, embeddings):
    """Test various symptom combinations against the loaded FAISS indices"""
    
    # Sample symptom combinations to test (based on actual disease_symptoms.csv data)
    test_cases = [
        # Respiratory/Cold-like symptoms
        ["cough", "chest pain", "breathlessness"],
        ["continuous sneezing", "watering from eyes", "throat irritation"],
        
        # Digestive issues
        ["stomach pain", "nausea", "vomiting"],
        ["loss of appetite", "dehydration", "weakness of one body side"],
        
        # Fever/Infection patterns  
        ["high fever", "chills", "fatigue"],
        ["headache", "muscle weakness", "sweating"],
        
        # Skin conditions
        ["itching", "skin rash", "redness of eyes"],
        ["skin rash", "joint pain", "skin peeling"],
        
        # Neurological symptoms
        ["headache", "dizziness", "blurred and distorted vision"],
        ["anxiety", "restlessness", "irritability"],
        
        # Urinary/Kidney issues
        ["burning micturition", "bladder discomfort", "foul smell of urine"],
        
        # Cardiovascular symptoms
        ["chest pain", "fast heart rate", "breathlessness"]
    ]
    
    print("="*60)
    print("TESTING SYMPTOM COMBINATIONS ON LOADED FAISS INDICES")
    print("="*60)
    
    for i, symptoms in enumerate(test_cases, 1):
        print(f"\nTest {i}: {', '.join(symptoms)}")
        
        # Create query string from symptoms
        query_text = ", ".join(symptoms)
        
        try:
            # Search symptom index using LangChain's similarity_search_with_score
            symptom_results = faiss_symptom.similarity_search_with_score(query_text, k=3)
            
            print(f"  Symptom Index Results:")
            for j, (doc, score) in enumerate(symptom_results):
                print(f"    {j+1}. Score: {score:.4f} | Content: {doc.page_content[:100]}...")
                if hasattr(doc, 'metadata') and doc.metadata:
                    print(f"       Metadata: {doc.metadata}")
            
        except Exception as e:
            print(f"  Symptom Index Error: {e}")
        
        try:
            # Search severity index
            severity_results = faiss_severity.similarity_search_with_score(query_text, k=3)
            
            print(f"  Severity Index Results:")
            for j, (doc, score) in enumerate(severity_results):
                print(f"    {j+1}. Score: {score:.4f} | Content: {doc.page_content[:100]}...")
                if hasattr(doc, 'metadata') and doc.metadata:
                    print(f"       Metadata: {doc.metadata}")
                    
        except Exception as e:
            print(f"  Severity Index Error: {e}")
        
        print("-" * 40)

def quick_symptom_search(faiss_index, symptoms, index_name="Index", k=5):
    """Quick search function for individual symptom queries"""
    query_text = ", ".join(symptoms) if isinstance(symptoms, list) else symptoms
    
    try:
        results = faiss_index.similarity_search_with_score(query_text, k=k)
        
        print(f"\n{index_name} Results for: '{query_text}'")
        for i, (doc, score) in enumerate(results):
            print(f"  {i+1}. Score: {score:.4f}")
            print(f"     Content: {doc.page_content}")
            if hasattr(doc, 'metadata') and doc.metadata:
                print(f"     Metadata: {doc.metadata}")
                
        return results
        
    except Exception as e:
        print(f"Error searching {index_name}: {e}")
        return []

def compare_indices(faiss_symptom, faiss_severity, query_symptoms):
    """Compare results between symptom and severity indices for the same query"""
    
    print(f"\n{'='*60}")
    print(f"COMPARING INDICES FOR: {', '.join(query_symptoms)}")
    print(f"{'='*60}")
    
    # Search both indices
    symptom_results = quick_symptom_search(faiss_symptom, query_symptoms, "Symptom Index", k=3)
    severity_results = quick_symptom_search(faiss_severity, query_symptoms, "Severity Index", k=3)
    
    # Simple comparison
    if symptom_results and severity_results:
        print(f"\nComparison:")
        print(f"  Best Symptom Match Score: {symptom_results[0][1]:.4f}")
        print(f"  Best Severity Match Score: {severity_results[0][1]:.4f}")
        
        symptom_content = symptom_results[0][0].page_content
        severity_content = severity_results[0][0].page_content
        
        print(f"  Symptom Index Best: {symptom_content[:80]}...")
        print(f"  Severity Index Best: {severity_content[:80]}...")

def batch_test_individual_symptoms(faiss_symptom, faiss_severity):
    """Test individual symptoms to see how the indices respond"""
    
    individual_symptoms = [
        "fever", "headache", "cough", "nausea", "chest pain",
        "dizziness", "fatigue", "skin rash", "vomiting", "anxiety"
    ]
    
    print(f"\n{'='*60}")
    print("TESTING INDIVIDUAL SYMPTOMS")
    print(f"{'='*60}")
    
    for symptom in individual_symptoms:
        print(f"\nTesting: '{symptom}'")
        
        # Get top result from each index
        try:
            sym_result = faiss_symptom.similarity_search_with_score(symptom, k=1)[0]
            sev_result = faiss_severity.similarity_search_with_score(symptom, k=1)[0]
            
            print(f"  Symptom: {sym_result[1]:.4f} | {sym_result[0].page_content[:50]}...")
            print(f"  Severity: {sev_result[1]:.4f} | {sev_result[0].page_content[:50]}...")
            
        except Exception as e:
            print(f"  Error: {e}")

# Main execution function
def run_all_tests(faiss_symptom, faiss_severity, embeddings):
    """Run all tests using your loaded FAISS indices"""
    
    print("Starting comprehensive symptom testing...")
    print(f"Symptom Index: {faiss_symptom}")
    print(f"Severity Index: {faiss_severity}")
    
    # Run the main symptom combination tests
    test_symptom_combinations(faiss_symptom, faiss_severity, embeddings)
    
    # Test individual symptoms
    batch_test_individual_symptoms(faiss_symptom, faiss_severity)
    
    # Example comparison
    compare_indices(faiss_symptom, faiss_severity, ["fever", "headache", "fatigue"])
    
    print(f"\n{'='*60}")
    print("ALL TESTS COMPLETED")
    print(f"{'='*60}")

# Usage:
# Assuming you have already loaded your indices as shown in your code:
run_all_tests(faiss_symptom, faiss_severity, embeddings)

Starting comprehensive symptom testing...
Symptom Index: <langchain_community.vectorstores.faiss.FAISS object at 0x00000194A95E2950>
Severity Index: <langchain_community.vectorstores.faiss.FAISS object at 0x00000194A9F2B650>
TESTING SYMPTOM COMBINATIONS ON LOADED FAISS INDICES

Test 1: cough, chest pain, breathlessness
  Symptom Index Results:
    1. Score: 0.2225 | Content: vomiting, breathlessness, chest pain...
       Metadata: {'disease': 'heart attack'}
    2. Score: 0.3563 | Content: breathlessness, sweating, chest pain...
       Metadata: {'disease': 'heart attack'}
    3. Score: 0.4396 | Content: vomiting, breathlessness, sweating, chest pain...
       Metadata: {'disease': 'heart attack'}
  Severity Index Results:
    1. Score: 0.5518 | Content: chest pain...
    2. Score: 0.6531 | Content: breathlessness...
    3. Score: 1.1602 | Content: stomach pain...
----------------------------------------

Test 2: continuous sneezing, watering from eyes, throat irritation
  Symptom Inde